In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cifar-10/trainLabels.csv
/kaggle/input/cifar-10/sampleSubmission.csv
/kaggle/input/cifar-10/test.7z
/kaggle/input/cifar-10/train.7z


In [2]:
pip install py7zr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 33.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.4/96.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.3/141.3 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.9/412.9 kB 17.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
!mkdir -p pipeline models data submissions

In [2]:
%%writefile pipeline/const.py
import os

BATCH_SIZE = 32
NUM_WORKERS = os.cpu_count()
CLASSES = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]
EPOCHS = 5
LEARNING_RATE = 0.001


Writing pipeline/const.py


In [5]:
import py7zr
# Đường dẫn đến các file cần giải nén
train_file = '/kaggle/input/cifar-10/train.7z'
test_file = '/kaggle/input/cifar-10/test.7z'

# Đường dẫn thư mục để giải nén
train_output_dir = '/kaggle/working/data/train/'
test_output_dir = '/kaggle/working/data/test'

# Giải nén train.7z
with py7zr.SevenZipFile(train_file, mode='r') as z:
    z.extractall(path=train_output_dir)
print("Giải nén thành công train.7z")

# Giải nén test.7z
with py7zr.SevenZipFile(test_file, mode='r') as z:
    z.extractall(path=test_output_dir)
print("Giải nén thành công test.7z")

print("DONE!")


Giải nén thành công train.7z
Giải nén thành công test.7z
DONE!


In [6]:
import os
kaggle_path = "/kaggle/working"
data_dir = os.path.join(kaggle_path, "data")

# Đường dẫn thư mục
train_dir = os.path.join(data_dir, "train", "train")
test_dir = os.path.join(data_dir, "test", "test")

# Đếm số tệp trong thư mục train
num_train_files = len([f for f in os.listdir(train_dir) if os.path.isfile(os.path.join(train_dir, f))])
print(f"Số tệp trong thư mục train: {num_train_files}")

# Đếm số tệp trong thư mục test
num_test_files = len([f for f in os.listdir(test_dir) if os.path.isfile(os.path.join(test_dir, f))])
print(f"Số tệp trong thư mục test: {num_test_files}")


Số tệp trong thư mục train: 50000
Số tệp trong thư mục test: 300000


In [7]:
train_labels = pd.read_csv("/kaggle/input/cifar-10/trainLabels.csv")
sample_submissions = pd.read_csv("/kaggle/input/cifar-10/sampleSubmission.csv")

In [3]:
%%writefile pipeline/data_setup.py
import os
import torch
import pathlib
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
import pandas as pd
from PIL import Image
from typing import Tuple

from pipeline import const

import importlib
importlib.reload(const)

NUM_WORKERS = const.NUM_WORKERS
BATCH_SIZE = const.BATCH_SIZE
CLASSES = const.CLASSES

# Write a custom dataset class (inherits from torch.utils.data.Dataset)
class_to_idx = {cls_name: i for i, cls_name in enumerate(CLASSES)}
# 1. Subclass torch.utils.data.Dataset
class TrainDatasetCustom(Dataset):
    
    # 2. Initialize with a targ_dir and transform (optional) parameter
    def __init__(self, targ_dir: str, labels: pd.DataFrame, transform=None) -> None:
        
        # 3. Create class attributes
        # Get all .png images and sort them by filename (numerically)
        self.paths = sorted(pathlib.Path(targ_dir).glob("*.png"), key=lambda x: int(x.stem))
        # Setup transforms
        self.transform = transform
        # Create classes and class_to_idx attributes
        self.classes, self.class_to_idx = CLASSES, class_to_idx
        # Set up labels
        self.labels = labels

    # 4. Make function to load images
    def load_image(self, index: int) -> Image.Image:
        "Opens an image via a path and returns it."
        image_path = self.paths[index]
        return Image.open(image_path) 
    
    # 5. Overwrite the __len__() method (optional but recommended for subclasses of torch.utils.data.Dataset)
    def __len__(self) -> int:
        "Returns the total number of samples."
        return len(self.paths)
    
    # 6. Overwrite the __getitem__() method (required for subclasses of torch.utils.data.Dataset)
    def __getitem__(self, index: int) -> Tuple[torch.Tensor, int]:
        "Returns one sample of data, data and label (X, y)."
        img = self.load_image(index)
        class_name  = self.labels["label"][index]
        class_idx = self.class_to_idx[class_name]

        # Transform if necessary
        if self.transform:
            return self.transform(img), class_idx # return data, label (X, y)
        else:
            return img, class_idx # return data, label (X, y)

# 1. Subclass torch.utils.data.Dataset
class TestDatasetCustom(Dataset):
    
    # 2. Initialize with a targ_dir and transform (optional) parameter
    def __init__(self, targ_dir: str, labels: pd.DataFrame, transform=None) -> None:
        
        # 3. Create class attributes
        # Get all .png images and sort them by filename (numerically)
        self.paths = sorted(pathlib.Path(targ_dir).glob("*.png"), key=lambda x: int(x.stem))
        # Setup transforms
        self.transform = transform
        # Create classes and class_to_idx attributes
        self.classes, self.class_to_idx = CLASSES, class_to_idx
        # Set up labels
        self.labels = labels

    # 4. Make function to load images
    def load_image(self, index: int) -> Image.Image:
        "Opens an image via a path and returns it."
        image_path = self.paths[index]
        return Image.open(image_path) 
    
    # 5. Overwrite the __len__() method (optional but recommended for subclasses of torch.utils.data.Dataset)
    def __len__(self) -> int:
        "Returns the total number of samples."
        return len(self.paths)
    
    # 6. Overwrite the __getitem__() method (required for subclasses of torch.utils.data.Dataset)
    def __getitem__(self, index: int) -> Tuple[torch.Tensor, int]:
        image_id = int(self.labels.iloc[index]['id'])
        image_path = self.paths[index]
        img = Image.open(image_path).convert("RGB")

        if self.transform:
            img = self.transform(img)

        return img, image_id  # trả về ảnh và id



def create_dataloaders(
    train_dir: str, 
    test_dir: str, 
    train_transform: transforms.Compose, 
    test_transform: transforms.Compose,
    train_labels: pd.DataFrame,
    test_labels: pd.DataFrame,
    batch_size: int, 
    num_workers: int=NUM_WORKERS
):
    
    # Use TrainDatasetCustom to create dataset(s)
    train_data_custom = TrainDatasetCustom(targ_dir=train_dir,
                                      labels=train_labels,
                                      transform=train_transform)
    test_data_custom = TestDatasetCustom(targ_dir=test_dir,
                                      labels=test_labels,
                                      transform=test_transform)
    
    # Get class names
    class_names = train_data_custom.classes
    
    # Turn images into data loaders
    train_dataloader = DataLoader(
      train_data_custom,
      batch_size=batch_size,
      shuffle=True,
      num_workers=num_workers,
      pin_memory=True,
    )
    
    test_dataloader = DataLoader(
      test_data_custom,
      batch_size=batch_size,
      shuffle=False, # don't need to shuffle test data
      num_workers=num_workers,
      pin_memory=True,
    )

    return train_dataloader, test_dataloader, class_names

Writing pipeline/data_setup.py


In [84]:
# Import data_setup.py
from pipeline import data_setup
from torchvision import transforms
from pipeline import const

import importlib
importlib.reload(data_setup)  # ép Python load lại code mới từ file

BATCH_SIZE = const.BATCH_SIZE
NUM_WORKERS = const.NUM_WORKERS


# Augment train data
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor()
])

# Don't augment test data, only reshape
test_transform = transforms.Compose([
    transforms.ToTensor()
])

# Create train/test dataloader and get class names as a list
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(train_dir=train_dir,
                                                                               test_dir=test_dir,
                                                                               train_transform=train_transform,
                                                                               test_transform=test_transform,
                                                                               train_labels=train_labels,
                                                                               test_labels=sample_submissions,
                                                                               batch_size=BATCH_SIZE,
                                                                               num_workers=NUM_WORKERS)
train_dataloader, test_dataloader

(<torch.utils.data.dataloader.DataLoader at 0x7a02252f0690>,
 <torch.utils.data.dataloader.DataLoader at 0x7a0229adbbd0>)

In [4]:
%%writefile pipeline/model_builder.py
"""
Contains PyTorch model code to instantiate an EfficientNetB0 model.
"""
import torch
from torch import nn
import torchvision.models as models

class EfficientNetB0(nn.Module):
    """Creates an EfficientNetB0 architecture for transfer learning.

    Args:
        output_shape: An integer indicating number of output classes.
        pretrained: Whether to use pretrained weights on ImageNet.
    """
    def __init__(self, in_features: int, output_shape: int, pretrained: bool = True) -> None:
        super().__init__()
        # Load EfficientNetB0 pretrained on ImageNet1K
        weights = models.EfficientNet_B0_Weights.DEFAULT if pretrained else None
        self.model = models.efficientnet_b0(weights=weights)

        # Mở toàn bộ trọng số (unfreeze) để fine-tune full model
        for param in self.model.parameters():
            param.requires_grad = True

        # Thay classifier cuối cùng bằng lớp Linear mới
        self.model.classifier = torch.nn.Sequential(
            torch.nn.Dropout(p=0.2, inplace=True), 
            torch.nn.Linear(in_features=in_features, 
                            out_features=output_shape, # same number of output units as our number of classes
                            bias=True))

    def forward(self, x: torch.Tensor):
        return self.model(x)


Writing pipeline/model_builder.py


In [25]:
import torch
# Import model_builder.py
from pipeline import model_builder
from pipeline import const
importlib.reload(model_builder)
CLASSES = const.CLASSES
device = "cuda" if torch.cuda.is_available() else "cpu"

# Instantiate an instance of the model from the "model_builder.py" script
torch.manual_seed(42)
model = model_builder.EfficientNetB0(in_features=1280,
                                        pretrained = True,
                                          output_shape=len(CLASSES)).to(device)

In [5]:
%%writefile pipeline/engine.py
"""
Contains functions for training and testing a PyTorch model.
"""
import torch

from tqdm.auto import tqdm
from typing import Dict, List, Tuple

def train_step(model: torch.nn.Module, 
               dataloader: torch.utils.data.DataLoader, 
               loss_fn: torch.nn.Module, 
               optimizer: torch.optim.Optimizer,
               device: torch.device) -> Tuple[float, float]:
    # Put model in train mode
    model.train()
    
    # Setup train loss and train accuracy values
    train_loss, train_acc = 0, 0
    
    # Loop through data loader data batches
    for batch, (X, y) in enumerate(dataloader):
        # Send data to target device
        X, y = X.to(device), y.to(device)

        # 1. Forward pass
        y_pred = model(X)

        # 2. Calculate  and accumulate loss
        loss = loss_fn(y_pred, y)
        train_loss += loss.item() 

        # 3. Optimizer zero grad
        optimizer.zero_grad()

        # 4. Loss backward
        loss.backward()

        # 5. Optimizer step
        optimizer.step()

        # Calculate and accumulate accuracy metrics across all batches
        y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
        train_acc += (y_pred_class == y).sum().item()/len(y_pred)

    # Adjust metrics to get average loss and accuracy per batch 
    train_loss = train_loss / len(dataloader)
    train_acc = train_acc / len(dataloader)
    return train_loss, train_acc

# 1. Take in various parameters required for training and test steps
def train(model: torch.nn.Module, 
          train_dataloader: torch.utils.data.DataLoader, 
          optimizer: torch.optim.Optimizer,
          device: torch.device,
          loss_fn: torch.nn.Module = torch.nn.CrossEntropyLoss(),
          epochs: int = 5) -> Dict[str, List]:
    
    # 2. Create empty results dictionary
    results = {"train_loss": [],
        "train_acc": []
    }
    
    # 3. Loop through training and testing steps for a number of epochs
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model=model,
                                           device=device,
                                           dataloader=train_dataloader,
                                           loss_fn=loss_fn,
                                           optimizer=optimizer)
        
        # 4. Print out what's happening
        print(
            f"Epoch: {epoch+1} | "
            f"train_loss: {train_loss:.4f} | "
            f"train_acc: {train_acc:.4f} | "
        )

        # 5. Update results dictionary
        # Ensure all data is moved to CPU and converted to float for storage
        results["train_loss"].append(train_loss.item() if isinstance(train_loss, torch.Tensor) else train_loss)
        results["train_acc"].append(train_acc.item() if isinstance(train_acc, torch.Tensor) else train_acc)

    # 6. Return the filled results at the end of the epochs
    return results

Writing pipeline/engine.py


In [88]:
from pipeline import engine
from pipeline import const
from torch import nn

import importlib
importlib.reload(const)
importlib.reload(engine)

EPOCHS = const.EPOCHS
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)

# Use train() by calling it from engine.py
engine.train(model=model,
             train_dataloader=train_dataloader,
             optimizer=optimizer,
             loss_fn=loss_fn,
             epochs=EPOCHS,
             device=device)

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.9285 | train_acc: 0.2957 | 
Epoch: 2 | train_loss: 1.6025 | train_acc: 0.4217 | 
Epoch: 3 | train_loss: 1.4946 | train_acc: 0.4624 | 
Epoch: 4 | train_loss: 1.4463 | train_acc: 0.4801 | 
Epoch: 5 | train_loss: 1.4219 | train_acc: 0.4916 | 


{'train_loss': [1.9285157425466135,
  1.6025101368959638,
  1.4946407941542446,
  1.4463360762260544,
  1.421942500563211],
 'train_acc': [0.295685380678183,
  0.42174504158669224,
  0.46243202175303905,
  0.4800663787587972,
  0.49164267434420983]}

In [6]:
%%writefile pipeline/utils.py
"""
Contains various utility functions for PyTorch model training and saving.
"""
import torch
from pathlib import Path

def save_model(model: torch.nn.Module,
               target_dir: str,
               model_name: str):
  """Saves a PyTorch model to a target directory.

  Args:
    model: A target PyTorch model to save.
    target_dir: A directory for saving the model to.
    model_name: A filename for the saved model. Should include
      either ".pth" or ".pt" as the file extension.
  
  Example usage:
    save_model(model=model_0,
               target_dir="models",
               model_name="05_going_modular_tingvgg_model.pth")
  """
  # Create target directory
  target_dir_path = Path(target_dir)
  target_dir_path.mkdir(parents=True,
                        exist_ok=True)
  
  # Create model save path
  assert model_name.endswith(".pth") or model_name.endswith(".pt"), "model_name should end with '.pt' or '.pth'"
  model_save_path = target_dir_path / model_name

  # Save the model state_dict()
  print(f"[INFO] Saving model to: {model_save_path}")
  torch.save(obj=model.state_dict(),
             f=model_save_path)

Writing pipeline/utils.py


In [90]:
# Import utils.py
from pipeline import utils

# Save a model to file
utils.save_model(model=model,
           target_dir="models",
           model_name="example_model.pth")

[INFO] Saving model to: models/example_model.pth


In [94]:
from pipeline import model_builder

loaded_model = model_builder.TinyVGG(
    input_shape=3,
    hidden_units=10,
    output_shape=len(CLASSES)
).to(device)

# Load trọng số
model_path = "models/example_model.pth"
loaded_model.load_state_dict(torch.load(model_path, map_location="cpu"))

<All keys matched successfully>

In [7]:
%%writefile pipeline/predict.py
from pipeline import model_builder
import torch
import pandas as pd

from pipeline import const
CLASSES = const.CLASSES
class_to_idx = {cls_name: i for i, cls_name in enumerate(CLASSES)}
idx_to_class = {v: k for k, v in class_to_idx.items()}

def predict_testset(model: torch.nn.Module,
                    submission_number: int,
                    submission_dir: str,
                    test_dataloader: torch.utils.data.DataLoader,
                    device: torch.device):
    submission_data = []
    
    model.eval()
    with torch.inference_mode():
        for imgs, image_ids in test_dataloader:
            outputs = model(imgs.to(device))
            preds = torch.argmax(torch.softmax(outputs, dim=1), dim=1)
    
            for img_id, pred in zip(image_ids, preds):
                predicted_label = idx_to_class[pred.item()]
                submission_data.append({'id': int(img_id), 'label': predicted_label})
                if(int(img_id) % 10000 == 0):
                    print(f"Done ảnh {img_id}")
                    
    # Convert the submission data to a DataFrame
    submission_df = pd.DataFrame(submission_data)
    
    # Save the submission file
    submission_df.to_csv(f'{submission_dir}/submission{submission_number}.csv', index=False)
    
    print(f"Submission file saved to 'submission{submission_number}.csv'.")
    

Writing pipeline/predict.py


In [98]:
from pipeline import predict
import importlib
importlib.reload(predict)

submission_dir = os.path.join(kaggle_path, "submissions")

predict.predict_testset(model=loaded_model,
                submission_number=1,
                submission_dir=submission_dir,
                test_dataloader=test_dataloader,
                device=device)

Done ảnh 10000
Done ảnh 20000
Done ảnh 30000
Done ảnh 40000
Done ảnh 50000
Done ảnh 60000
Done ảnh 70000
Done ảnh 80000
Done ảnh 90000
Done ảnh 100000
Done ảnh 110000
Done ảnh 120000
Done ảnh 130000
Done ảnh 140000
Done ảnh 150000
Done ảnh 160000
Done ảnh 170000
Done ảnh 180000
Done ảnh 190000
Done ảnh 200000
Done ảnh 210000
Done ảnh 220000
Done ảnh 230000
Done ảnh 240000
Done ảnh 250000
Done ảnh 260000
Done ảnh 270000
Done ảnh 280000
Done ảnh 290000
Done ảnh 300000
Submission file saved to 'submission1.csv'.


In [8]:
%%writefile pipeline/train.py
"""
Trains a PyTorch image classification model using device-agnostic code.
"""

import os
import torch
from pipeline import data_setup, engine, model_builder, utils, const, predict
import pandas as pd

from torchvision import transforms

import importlib
importlib.reload(data_setup)
importlib.reload(engine)
importlib.reload(model_builder)
importlib.reload(utils)
importlib.reload(const)
importlib.reload(predict)

# Setup hyperparameters
EPOCHS = const.EPOCHS
BATCH_SIZE = const.BATCH_SIZE
CLASSES = const.CLASSES
HIDDEN_UNITS = len(CLASSES)
LEARNING_RATE = const.LEARNING_RATE
NUM_WORKERS = const.NUM_WORKERS

# Setup directories
kaggle_path = "/kaggle/working"
data_dir = os.path.join(kaggle_path, "data")

# Đường dẫn thư mục
train_dir = os.path.join(data_dir, "train", "train")
test_dir = os.path.join(data_dir, "test", "test")

# Setup target device
device = "cuda" if torch.cuda.is_available() else "cpu"

# DataFrame
train_labels = pd.read_csv("/kaggle/input/cifar-10/trainLabels.csv")
sample_submissions = pd.read_csv("/kaggle/input/cifar-10/sampleSubmission.csv")

# Create transform
train_transform = transforms.Compose([
    transforms.Resize((224, 224)), # 1. Reshape all images to 224x224 (though some models may require different sizes)
    transforms.ToTensor(), # 2. Turn image values to between 0 & 1 
    transforms.Normalize(mean=[0.485, 0.456, 0.406], # 3. A mean of [0.485, 0.456, 0.406] (across each colour channel)
                         std=[0.229, 0.224, 0.225]) # 4. A standard deviation of [0.229, 0.224, 0.225] (across each colour channel),
])
test_transform = transforms.Compose([
    transforms.Resize((224, 224)), # 1. Reshape all images to 224x224 (though some models may require different sizes)
    transforms.ToTensor(), # 2. Turn image values to between 0 & 1 
    transforms.Normalize(mean=[0.485, 0.456, 0.406], # 3. A mean of [0.485, 0.456, 0.406] (across each colour channel)
                         std=[0.229, 0.224, 0.225]) # 4. A standard deviation of [0.229, 0.224, 0.225] (across each colour channel),
])

# Create train/test dataloader and get class names as a list
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(train_dir=train_dir,
                                                                               test_dir=test_dir,
                                                                               train_transform=train_transform,
                                                                               test_transform=test_transform,
                                                                               train_labels=train_labels,
                                                                               test_labels=sample_submissions,
                                                                               batch_size=BATCH_SIZE,
                                                                               num_workers=NUM_WORKERS)

# Create model with help from model_builder.py
model = model_builder.EfficientNetB0(in_features=1280,
                                        pretrained = True,
                                          output_shape=len(CLASSES)).to(device)

# Set loss and optimizer
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),
                             lr=LEARNING_RATE)

# Start training with help from engine.py
engine.train(model=model,
             train_dataloader=train_dataloader,
             optimizer=optimizer,
             loss_fn=loss_fn,
             epochs=EPOCHS,
             device=device)

# Save the model with help from utils.py
utils.save_model(model=model,
           target_dir=os.path.join(kaggle_path, "models"),
           model_name="example_model.pth")

loaded_model = model_builder.EfficientNetB0(
    in_features=1280,
    pretrained = True,
    output_shape=len(CLASSES)).to(device)         

# Load trọng số
model_path = os.path.join(kaggle_path, "models", "example_model.pth")
loaded_model.load_state_dict(torch.load(model_path, map_location=device))

# Predict và lưu vào file submission
submission_dir = os.path.join(kaggle_path, "submissions")

predict.predict_testset(model=loaded_model,
                        submission_number=1,
                        submission_dir=submission_dir,
                        test_dataloader=test_dataloader,
                        device=device)

Writing pipeline/train.py


In [10]:
%%writefile pipeline/main.py

import os
import py7zr
import torch

def main():
    # 1. Cài thư mục cần thiết
    os.makedirs("pipeline", exist_ok=True)
    os.makedirs("models", exist_ok=True)
    os.makedirs("data", exist_ok=True)
    os.makedirs("submissions", exist_ok=True)
    
    # 2. Đường dẫn đến file .7z
    train_file = '/kaggle/input/cifar-10/train.7z'
    test_file  = '/kaggle/input/cifar-10/test.7z'
    
    train_output_dir = '/kaggle/working/data/train/'
    test_output_dir  = '/kaggle/working/data/test/'
    
    # 3. Giải nén dữ liệu train
    with py7zr.SevenZipFile(train_file, mode='r') as z:
        z.extractall(path=train_output_dir)
    print("[INFO] Giải nén thành công train.7z")
    
    # 4. Giải nén dữ liệu test
    with py7zr.SevenZipFile(test_file, mode='r') as z:
        z.extractall(path=test_output_dir)
    print("[INFO] Giải nén thành công test.7z")
    
    print("[INFO] DONE extract!")
    
    # 5. Gọi script train.py trong pipeline
    print("[INFO] Bắt đầu train...")
    os.system("python -m pipeline.train")
if __name__ == "__main__":
    main()


Overwriting pipeline/main.py


### CHẠY TẤT CẢ CÁC CELL TẠO FILE VÀ CHẠY CÁC DÒNG CODE SAU

In [11]:
!touch pipeline/__init__.py
!pip install py7zr
!python pipeline/main.py

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 38.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.4/96.4 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.3/141.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.9/412.9 kB 22.5 MB/s eta 0:00:00
[INFO] Giải nén thành công train.7z
[INFO] Giải nén thành công test.7z
[INFO] DONE extract!
[INFO] Bắt đầu train...
Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth
100%|███████████████████████████████████████| 20.5M/20.5M [00:00<00:00, 119MB/s]
100%|████████████████████████████████████████████| 5/5 [20:24<00:00, 244.99s/it]
[INFO] Saving model to: /kaggle/working/models/example_model.pth
Done ảnh